![Opendoor](https://raw.githubusercontent.com/interviewquery/takehomes/opendoor_1/opendoor_1/logo.png)
# Opendoor Data Science Take Home Problem Set

The questions below are meant to give candidates a sense of the problems
that we tackle at Opendoor. We expect solutions in the form of a
jupyter notebook + working code. The problem set should take around 3 hours to
complete.

# Part 1. Simple housing model 

Your task is to implement a simple model to predict home prices for a
small real estate transactions dataset.

## Instructions

1.  Load the data in `data.csv`

2.  Predict the close price as of list date given the other attributes.

3.  Build separate models with and without ListPrice.

4.  Feel free to join the dataset to any other data sources, so long as
    they are not leaky.

## Questions

1.  Describe your methodology. Why did you pick it?

2.  How well would you do if you excluded the list price?

3.  What is the performance of your model? What error metrics did you
    choose?

4.  How would you improve your model?

5.  How would you host your model in a production environment to predict
    values of homes in real-time?

# Part 2. Simple ​$n$-grams

Generate the top 10 trigrams for the article
[http://en.wikipedia.org/wiki/N-gram](http://en.wikipedia.org/wiki/N-gram)

# Part 3. Memoizer 

Write a function that accepts a single-argument function ​`f`​, and an
integer `k`​ ​, and returns a function that behaves the same as ​`f`​ except
it caches the last ​`k`​ distinct accessed results of ​`f`​.

For instance, if ​`memoize`​ is the function we're after, and let ​`mem_f =
memoize(f, 2)`, then

-   `mem_f(arg1)​` → `f(arg1)​` ​ is computed and cached

-   `mem_f(arg1)` → `f(arg1)` is returned from cache

-   `mem_f(arg2)` → `f(arg2)` ​is computed and cached

-   `mem_f(arg3)` → `f(arg3)` ​is computed and cached, and ​`f(arg1)​` is
    evicted

## Additional questions

-   Can you describe the efficiency of the memoizer?

-   How does your memoizer handle concurrent access?


In [1]:
# Part 1
import numpy as np
from dateutil.parser import parse
import heapq
from statistics import median
from functools import total_ordering

@total_ordering
class Home(object):
	def __init__(self, lat, lng, close, price):
		self.lat = float(lat)
		self.long = float(lng)
		self.close_date = parse(close) # convert to dateutil object for date comparison
		self.close_price = float(price)

	def __str__(self):
		return "lat: {} long: {} close_date: {} close_price: {}".format(self.lat, self.long, self.close_date, self.close_price)

	def __eq__(self, other):
		# homes are equal if they have same lat, lng, close price, close date
		return self.lat == other.lat and self.long == other.long and self.close_date == other.close_date and self.close_price == other.close_price

	def __lt__(self, other):
		return self.close_price < other.close_price

def distance(home1, home2):
	# returns the l2 distance
	return np.sqrt((home1.lat - home2.lat)**2 + (home1.long - home2.long)**2)

def softmax(v):
        """Calculates the softmax function that outputs a vector of values that sum to one.
        Softmax is np.exp(v) / sum(np.exp(v)), but this could present numerical overflow issues if v is very large. 
        We can work around this by adding a constant into the exponent. 
        This is because Ce^t = e^(t + logC)
        """
        v = np.array(v)
        logC = -np.max(v)
        return np.exp(v + logC)/np.sum(np.exp(v + logC), axis = 0)


def weight_homes(target_home, other_homes):
	# assigns weights to homes based on the target homes. 
	# takes the distances from target homes as the weights, but then runs softmax over the resulting vector to normalize to one. 
	# with more data about the homes, we would want to use a similarity_score(home1, home2) function here, instead of only using the l2 distance.
	# For example, if two homes are approximately the same distance to the target home, but also has the same square footage,
	# then it should have a larger weight.
	return softmax([distance(target_home, home) for home in other_homes])

def is_valid_home(current_home, candidate_home):
	# the candidate home must not equal the current home. It also needs to to have a strictly earlier close date. 
	return candidate_home != current_home and candidate_home.close_date < current_home.close_date

def get_max_distance_candidate(current_home, candidate_homes):
	return max(candidate_homes, key = lambda home: distance(current_home, home))

def single_predict(current_home, data, k = 4):
	candidate_homes = []
	for d in data:
		if is_valid_home(current_home, d):
			if len(candidate_homes) < k:
				candidate_homes.append(d)
			else:
				cmax = get_max_distance_candidate(current_home, candidate_homes)
				if distance(current_home, d) < distance(current_home, cmax):
					candidate_homes.remove(cmax)
					candidate_homes.append(d)
	# sanity check: if a home is valid and not in the list of candidate homes, then its distance must be >= the maximum candidate home's distance to the current home.
	for d in data:
		assert not is_valid_home(current_home, d) or d in candidate_homes \
			or distance(d, current_home) >= distance(get_max_distance_candidate(current_home, candidate_homes), current_home), \
			"Sanity check failed, found home with a smaller distance diff that is not in the list."
	# sanity check: the candidate home should not include the current home, and all home close dates should be less than the current home's sold date
	if len(candidate_homes) == 0:
		# case where we picked the home with no prior sales before it - don't have any information to go off of
		return 0
	# note: finding k neighbors may be impossible in case there are < k houses in the dataset that closed before it. 
	# in this case, the price predicted is 0 if it is the single house with no houses sold before it. 
	# otherwise, we use the n nearest neighbors to predict, but we have n < k. 
	for c in candidate_homes:
		assert c != current_home and c.close_date < current_home.close_date, "Error: invalid home in candidate homes"
	# weight the homes according to their distance to the current home
	weights = weight_homes(current_home, candidate_homes)
	# get the price of the current home
	price = sum([weights[i] * candidate_homes[i].close_price for i in range(len(candidate_homes))])
	return price

def mrae(predicted_prices, actual_prices):
	errs = sorted([abs(predicted_prices[i] - actual_prices[i])/actual_prices[i]
		for i in range(len(predicted_prices))])
	return median(errs)

	
if __name__ == '__main__':
	print("reading data")
	with open('data.csv') as f:
		lines = f.readlines()
	print("done reading, now parsing data")
	homes = []
	for line in lines[1:]:
		lat, lng, close, price = line.strip().split(",")
		h = Home(lat, lng, close, price)
		homes.append(h)
	print("creating a subset of 5000 random homes to run predictions on")
	# take a subset of size 5000 random homes
	random_homes = [homes[np.random.randint(0, len(homes))] for _ in range(5000)]
	actual = [h.close_price for h in random_homes]
	preds = []
	for i in range(len(random_homes)):
		if i % 50 == 0:
			print("on prediction {}".format(i))
		preds.append(single_predict(random_homes[i], homes))
	print("done with 5000 predictions!")
	print("Median relative absolute error is {}".format(mrae(preds, actual)))






reading data
done reading, now parsing data
creating a subset of 5000 random homes to run predictions on
on prediction 0
on prediction 50
on prediction 100
on prediction 150
on prediction 200
on prediction 250
on prediction 300
on prediction 350
on prediction 400
on prediction 450
on prediction 500
on prediction 550
on prediction 600
on prediction 650
on prediction 700
on prediction 750
on prediction 800
on prediction 850
on prediction 900
on prediction 950
on prediction 1000
on prediction 1050
on prediction 1100
on prediction 1150
on prediction 1200
on prediction 1250
on prediction 1300
on prediction 1350
on prediction 1400
on prediction 1450
on prediction 1500
on prediction 1550
on prediction 1600
on prediction 1650
on prediction 1700
on prediction 1750
on prediction 1800
on prediction 1850
on prediction 1900
on prediction 1950
on prediction 2000
on prediction 2050
on prediction 2100
on prediction 2150
on prediction 2200
on prediction 2250
on prediction 2300
on prediction 2350
on pre

In [2]:
# Part 2, using wikipedia package
import wikipedia
import pandas as pd

n_gram_article = wikipedia.page("n-gram").content

def get_top_k_n_grams(text,n,k):
    word_list = text.split()
    n_grams = []
    freq_count = {}
    for i in range(len(word_list)-n-1):
        n_gram = ' '.join(word_list[i:i+n])
        n_grams.append(n_gram)
        if n_gram in freq_count.keys():
            freq_count[n_gram] += 1
        else:
            freq_count[n_gram] = 1
    top_n_gram_counts = sorted(freq_count.items(), key=lambda item: item[1], reverse=True)[:k]
    return [i[0] for i in top_n_gram_counts]

get_top_k_n_grams(n_gram_article,3,10)

['serve as the',
 'x i −',
 'part of the',
 'n-gram models are',
 'n − 1',
 '− 1 )',
 'the language model',
 'the probability of',
 'can also be',
 'also be used']

In [3]:
# Part 3
import numpy as np

def get_k_closest(li, item_idx, k):
	if k >= len(li):
		return li
	closest = []
	for i in range(item_idx - k, item_idx + k + 1):
		if i >= 0 and i < len(li) and i != item_idx:
			closest.append((li[i], abs(li[i] - li[item_idx])))
	return sorted(closest, key = lambda a: a[1])[:k]


if __name__ == '__main__':
	li = sorted([np.random.randint(100) for _ in range(100)])
	print(li[44])
	print(li[35:55])
	print(get_k_closest(li, 44, 4))

48
[37, 39, 40, 40, 40, 40, 43, 45, 47, 48, 48, 49, 49, 50, 52, 53, 56, 56, 56, 58]
[(48, 0), (47, 1), (49, 1), (49, 1)]
